# Measuring Hate Speech: Example 

This notebook details an example of training a network to predict the continuous hate speech score available in the Measuring Hate Speech corpus.

Content Warning: The data frame we download from HuggingFace contains profane, offensive, and hateful content.

In [ ]:
import datasets
import pandas as pd
import tensorflow as tf
import transformers

# The hate_measure repo must be locally installed
from hate_measure.nn import regressors

In [ ]:
# Download the Measuring Hate Speech corpus from HuggingFace
dataset = datasets.load_dataset('ucberkeley-dlab/measuring-hate-speech', 'binary')
df = dataset['train'].to_pandas()

In [ ]:
# Extract corresponding columns
x = df['text']
y = df['hate_speech_score'].values

In [ ]:
# Tokenize using RoBERTa Base
tokenizer = transformers.RobertaTokenizer.from_pretrained("roberta-base")
tokens = tokenizer(x.tolist(), return_tensors='np', padding=True)
inputs = [tokens['input_ids'], tokens['attention_mask']]

In [ ]:
# Build the hate speech measurer using the hate_measure repo
model = regressors.HateSpeechMeasurer.build_model(
    transformer='roberta-base',
    n_dense=64,
    max_length=tokens['input_ids'].shape[1],
    dropout_rate=0.1)

In [ ]:
# Compile using mean squared error loss
model.compile(
    optimizer=tf.keras.optimizers.Adam(lr=0.001, epsilon=1e-8),
    loss='mean_squared_error')

In [ ]:
# Train the model. This may take a while depending on your set-up.
history = model.fit(x=inputs,
                    y=y,
                    batch_size=16,
                    epochs=5,
                    verbose=True)